In [1]:
import os
import shutil
from sklearn.model_selection import train_test_split
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import pipeline

In [5]:
model_path = "../CLIP-GPT2/models/gpt2-small"

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.save_pretrained(model_path)

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.save_pretrained(model_path)

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [ ]:
## DATA PREPERATION
def file_to_list(text_loc, tokenizer, max_len):
    text_blocks = []
    
    f = open(text_loc, errors='ignore')
    text = f.read()
    
    tokens = tokenizer.encode(text)
    
    while len(tokens) > 0:
        holder = []
        if len(tokens) > max_len:
            holder = tokens[0:max_len]
            del tokens[0:max_len]
            if holder[-1] != 50256:
                holder.append(50256)
        
        else:
            holder = tokens
            tokens = []
            if holder[-1] != 50256:
                holder.append(50256)
                
        text_blocks.append(holder)
    
    return text_blocks

def text_to_pieces(text_loc, tokenizer, max_len=1024):
    text_blocks = file_to_list(text_loc, tokenizer, max_len)    
    return text_blocks

text_loc = "../datasets/AVA/AVA_captions_full.txt"
max_len = 1024

data = text_to_pieces(text_loc, tokenizer, max_len)

In [ ]:
train, test = train_test_split(data, test_size=0.1, random_state=42)

In [ ]:
train_loc = "../CLIP-GPT2/data/train.txt"
test_loc = "../CLIP-GPT2/data/test.txt"

In [ ]:
train_text = ""
train_text = train_text.join(train)

test_text = ""
test_text = test_text.join(test)

with open(train_loc, "x") as f:
    f.write(train_text)

with open(test_loc, "x") as f:
    f.write(test_text)

# Restart Runtime

In [ ]:
model_path = "../CLIP-GPT2/models/gpt2-small"
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

In [ ]:
def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=train_path,
        block_size=1024,
    )
    test_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=test_path,
        block_size=1024,
    )
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator